In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="-1"

In [51]:
import argparse
import os
import random
import signal
import sys
import uuid
from collections import deque

import cv2

import gym
import gym_donkeycar
import numpy as np
import tensorflow as tf
# from tensorflow.keras import backend as K
from tensorflow.keras.layers import Activation, Conv2D, Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import TimeDistributed as TD
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt # debug
# logging
# from stable_baselines import logger
# from stable_baselines.common import explained_variance, tf_util, TensorboardWriter
# from stable_baselines.common.tf_util import mse, total_episode_reward_logger
import math
import datetime
img_rows, img_cols = 80, 80
# Convert image into Black and white
img_channels = 4  # We stack 4 frames

In [57]:
model = tf.keras.applications.MobileNetV2(weights="imagenet")

In [58]:
model.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_5[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
_______________________________________________________________________________

In [7]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import glob
from PIL import Image

In [96]:
next_obs = cv2.imread("images/0_cam-image_array_.jpg")

In [97]:
next_obs.shape

(120, 160, 3)

In [61]:
next_obs = cv2.resize(next_obs, (224, 224))

(160, 120, 3)

In [19]:
input_img = tf.keras.applications.mobilenet_v2.preprocess_input(next_obs)

In [20]:
input_img.shape

(224, 224, 3)

In [23]:
type(input_img)

numpy.ndarray

In [27]:
input_img = input_img.reshape(1, input_img.shape[0], input_img.shape[1], input_img.shape[2])
input_img.shape

(1, 224, 224, 3)

In [46]:
x = model(input_img)

In [36]:
x.shape

TensorShape([1, 1000])

## My first transfer learning model with FC

In [133]:
feature_extractor_model = tf.keras.applications.MobileNetV2(weights="imagenet", include_top=False, input_shape=(96, 96, 3))

In [113]:
feature_extractor_model.summary()

Model: "mobilenetv2_1.00_96"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           [(None, 96, 96, 3)]  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 97, 97, 3)    0           input_12[0][0]                   
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 48, 48, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 48, 48, 32)   128         Conv1[0][0]                      
________________________________________________________________________________

In [105]:
next_obs = cv2.resize(next_obs, (96, 96))

In [106]:
np.max(next_obs)

255

In [107]:
np.min(next_obs)

52

In [142]:
input_img = tf.keras.applications.mobilenet_v2.preprocess_input(next_obs)

In [143]:
input_img.shape

(96, 96, 3)

In [144]:
input_img = input_img.reshape(1, input_img.shape[0], input_img.shape[1], input_img.shape[2])
input_img.shape

(1, 96, 96, 3)

In [110]:
print(np.max(input_img))
print(np.min(input_img))

1.0
-0.5921569


In [114]:
x = feature_extractor_model(input_img)
x.shape

TensorShape([1, 3, 3, 1280])

In [122]:
class DQNAgent:
    def __init__(self):
        self.learning_rate = 1e-4
    def build_model(self):
        model = Sequential()
        model.add(Input(shape=(3, 3, 1280*4), name='features_in')) # 4*80*80
        model.add(GlobalAveragePooling2D()) # replace the MobileNet head

        model.add(Dense(128, activation="relu"))
        
        # 15 categorical bins for Steering angles
        model.add(Dense(15, activation="linear"))

        adam = Adam(lr=self.learning_rate)
        model.compile(loss="mse", optimizer=adam)

        return model

In [123]:
dqn_agent = DQNAgent()
main_model = dqn_agent.build_model()
main_model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
global_average_pooling2d_9 ( (None, 5120)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 128)               655488    
_________________________________________________________________
dense_6 (Dense)              (None, 15)                1935      
Total params: 657,423
Trainable params: 657,423
Non-trainable params: 0
_________________________________________________________________


In [125]:
x_t = x
x_t.shape

TensorShape([1, 3, 3, 1280])

In [129]:
s_t = np.concatenate((x_t,)*img_channels, axis=3) # 80*80*4

In [130]:
s_t.shape

(1, 3, 3, 5120)

In [131]:
main_model(s_t)

<tf.Tensor: shape=(1, 15), dtype=float32, numpy=
array([[ 0.3021469 , -0.9252907 ,  2.7830405 ,  1.6227324 , -0.38740027,
         0.36568186, -1.6383058 , -1.4810058 ,  1.6717654 , -2.2445538 ,
        -0.9631863 , -1.3291183 ,  0.48429143,  0.09114917, -0.39772612]],
      dtype=float32)>

In [150]:
x_t.shape

TensorShape([1, 3, 3, 1280])

In [153]:
img_channels = 4
s_t1 = np.append(x_t, s_t[:, :, :, :((img_channels-1)*(x_t.shape[3]))], axis=3) # (1, 3, 3, 5120)  

In [154]:
s_t1.shape

(1, 3, 3, 5120)